error analysis in gr
error analysis in aharanov bohm stuff.
robotic knuematics
gyroscopes + float error or other error
hmm.

A bound on the length of the path does bound a loss.

Vectors on  tangent plane of spheres

The distance the base point moved
phi, theta -> phi, theta2

Abosulte value of curvature oculd be something like a lipshitz ocnstant


```python
import sympy
from dataclasses import dataclass

@dataclass
class ApproxEq():
    eps : Expr
    x : Expr
    y : Expr
    ctx : list[Expr] = field(default_factory=list) # list[ApproxEq] ? Hmm. both? Is eps a variable or expression in these? 
    def symm(self):
        return ApproxEq(self.eps, self.y, self.x)
    def __matmul__(self, other): # trans
        assert self.y == other.x
        return ApproxEq(self.eps + other.eps, self.x, other.z)
    @static_method
    def refl(e):
        return App
    @static_method
    def const(e : Expr):
        return ApproxEq( 10.0**(-digits),e , Numerical(e, digits))
    

p1 = ApproxEq(abs(x)**3, sin(x), x)

ApproxEq(1, x, ceil(x))

ApproxEq(0.01, sympy.pi, 3.14)
ApproxEq(0.01, sympy.E,  2.78)




```



Higher equalities as area/volume  errors?

Definitional vs propositional equality. Actual `=` vs `approxeq`. `=` is allowed to replace willy nilly

ind-eq : P(x)
eps1 : (forall x, R x x) -> eps2 : x = y -> eps3: R x y

R has to be lishitz 1? Would kind of make sense for indicator functions? No. That's not what lipshitz is. A principle to turn

Error tracking model (x, eps). ball arith a is bit like a subset model. Everything is a tuple of stuff

https://arxiv.org/abs/2501.14598 kellison type based approach to error

Approximate equality as a model of proof relevant equality.
It's kind of symmettric, kind of transitive but not on the nose

eq(pf, a, b)
approx_eq(eps, a, b)

eq(pf, a, b), eq(pf2, b, c) => eq(comp(pf1,pf2), a, c)
approx_eq(eps, a, b), approx_eq(eps2, b, c) -> approx_eq(eps1 + eps2, a, c)

Lipshitz as a kind of collapsing of structure



approx(eps, a, b) -> approx(L eps , f(a), f(b))
approx(eps, a, b) -> approx( L(a,b) eps )
approx(epx, f(a), f(b)) -> approx(eps / L, a, b) # if "injective"


approx(eps) epprox(eps) approx(eps) - lim eps -> 0  > eq()
                                   -> lim eps&2 -> 0 -> eq

There is something amusing about the connection of something as hgh falutin as proof relevance to something as crass a numerical bounds. But in a bishop sense, it also makes perfect sense.

A space is epsilon contractiable if everytinng is <= eps apart.

It could be reasonable to say that approx_eq(eps, a, b) isn't even a hting if a and b can't be connected by a path.
If eps is length of a geodesic

Path lifing
approxeq(path, a, b) where path : Real -> T an
approxeq(path,a,b) := is_cont_unit(path) /\ path(0) = a /\ path(1) = b

Fibers
Look at my randomized thing, my mind space last summer.



Screw actually taking the right float epilon. Do numerical analysis style kind of. Higham

Approximation rewriting - knuckled rewriting combinators. Run to possibly just get reasomable bound
Approximation completion
Approximation egraph
Approximation Calc - make Calc environment, but track epsilon
Approximation congruence closure - Lipshitz constants or interval propagators

approximate definition. `approx_add(x,y) :={eps} x + y`
flat equations

triangle inequality. any metric
|t - s| + |s - u| <= |t - u|


(x e+ y) e+ z ={4eps} x e+ (y e+ z) 

t ={eps} s
if defined as
|t - s| <= eps

Intuitively it makes sense.
Search becomes shittier.
Shortest path.
But reasonable spanning trees can also be ok.


There is something "higher D" like a 2 category or proof relevance that we may need to do rewriting on our symbolic errors also.
It's something like conditional rewriting.


ultrametrics?


Hamming
Higham
Wilkinson books
https://fpbench.org/education.html

LAProof https://mohittkr.github.io/LAProof.pdf

The rounding error of inner product example Higham 3.1




In [ ]:
from kdrag.all import *

approx_add = smt.Function("~add", smt.RealSort(),smt.RealSort(), smt.RealSort())

x,y,z, eps = smt.Reals("x y z eps")
add_defn = kd.axiom(smt.ForAll([x,y], 
    smt.Abs(approx_add(x,y) - (x + y)) <= 0.001 , patterns=[approx_add(x,y)]) 
    )
    # * smt.Abs(x + y)
add_comm = kd.axiom(smt.ForAll([x,y], approx_add(x,y) == approx_add(y,x), patterns=[approx_add(x,y)]))

approx = kd.define("approx", [x,y,eps], smt.Abs(x - y) <= eps ) #* smt.Abs(x)

kd.prove(approx(approx_add(x, approx_add(y,z)),
  approx_add(approx_add(x,y), z),
 0.003), unfold=2, by=[add_defn], timeout=5000)


       approx(x, y, eps) ==
       (If(x - y > 0, x - y, -(x - y)) <=
        eps*If(x > 0, x, -x))) to ForAll([x, y, eps],
       approx(x, y, eps) ==
       (If(x - y > 0, x - y, -(x - y)) <= eps))


TimeoutError: Timeout. Maybe you have given `prove` too many or not enough lemmas?

In [35]:
@kd.PTheorem(approx(approx_add(x, approx_add(y,z)),
  approx_add(approx_add(x,y), z),
 0.01))
def approx_add_assoc(l):
    l.unfold(approx)
    l.simp()
    l.have(approx(approx_add(x,y), x + y, 0.001), by=[add_defn, approx.defn])
    l.have(approx(approx_add(x,approx_add(y,z)), x + approx_add(y,z), 0.001), by=[add_defn, approx.defn])
    l.have(approx(approx_add(y,z), y + z, 0.001), by=[add_defn, approx.defn])
    l.have(approx(approx_add(x,y) + z, x + y + z, 0.001), by=[add_defn, approx.defn])
    l.unfold(approx, at=0)
    l.unfold(approx, at=1)
    l.unfold(approx, at=2)
    

Next Goal:
 [If(~add(x, y) - (x + y) > 0,
   ~add(x, y) - (x + y),
   -(~add(x, y) - (x + y))) <=
1/1000,
 If(~add(x, ~add(y, z)) - (x + ~add(y, z)) > 0,
   ~add(x, ~add(y, z)) - (x + ~add(y, z)),
   -(~add(x, ~add(y, z)) - (x + ~add(y, z)))) <=
1/1000,
 If(~add(y, z) - (y + z) > 0,
   ~add(y, z) - (y + z),
   -(~add(y, z) - (y + z))) <=
1/1000,
 approx(~add(x, y) + z, x + y + z, 1/1000)]
?|= If(~add(x, ~add(y, z)) + -1*~add(~add(x, y), z) <= 0,
   -1*~add(x, ~add(y, z)) + ~add(~add(x, y), z),
   ~add(x, ~add(y, z)) + -1*~add(~add(x, y), z)) <=
1/100


I think I maybe missed one of the sessions where the ideas were discussed in more detail.
But I think the idea of an approximation egraph is new and interesting to me.


Something I've been thinking about is the relationship of this to proofs or proof relevant equality. The appeal of such an analogy is to take fancy stuff people talk about 
One perspective of that of egraph proofs. Proofs are kind of retaining a spanning tree. Approximate equality wants to talk about shortest paths, but the shortest paths may not be in the minimal spanning tree.
approxeq(eps, x, y) := |x - y| <= epspfeq(p, x, y)
approxeq(eps1, x,y), -> approxeq(eps2,y,z) -> approxeq(eps1 + eps2, x, z)pfeq(p,x,y) -> pfeq(q, y,z) -> pfeq(comp(p,q), x, z)

something else people talk about is connectedness / path equality. Two points are on a connected shape if there is a continuous path between them.patheq(p, x, y) := continuous(p) /\ p[0] = x /\ p[1] = y
The length of a path is a property of it. In curved space


Approximation egraphs are interesting but what about just Approximation rewriting?
One could implement a rewriting system that tracks that accuracy loss as one rewrites, perhaps stopping when the accuracy loss becomes unacceptable.What about confluence of such a system? Different paths may of course give different bounds even if they end up at the same approximation.Approximation Rewriting feels like ap pragmatic approach to try and rewrite something into a simpler approximately equal form. I don't recall seeing this explored.

Weighted rewriting feels similar, but https://inria.hal.science/hal-05380979/document they don't have this use case in mind.




